# Tutorial 03: Model Asssumptions and Causality

#### Lecture and Tutorial Learning Goals:
After completing this week's lecture and tutorial work, you will be able to:

1. Give an example of a real problem that aims to test a causal relationship between variables.
2. Give an example of a real problem where the model can only establish an association between the response and the input variables.
3. Discuss how the desired goal of generative modelling is usually to make causal claims; however, we cannot often/easily do so (e.g., in particular, in the context of observational studies).
4. Discuss the role of confounders in causal inference.
5. Describe heteroscedasticity and the problem it presents to generative modeling.
6. Write a computer script to assess whether heteroscedasticity in a given data set, and if so, use practical solutions to manage it.
7. Describe colinearity and the problem it presents to generative modelling.
8. Write a computer script to assess whether collinearity exists between input variables in a given data set, and if so, use practical solutions to manage it.
9. Discuss what model diagnostics the data scientist can do by themselves and when the data scientist needs to consult a domain expert.

In [ ]:
# Run this cell before continuing.
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(GGally)
library(AER)
source("tests_tutorial_03.R")

## 1. Multicollinearity in Practice

In this tutorial, we will work with the California test score data `CASchools` from the package `AER` with $n = 420$. The [package's documentation](https://rdrr.io/cran/AER/man/CASchools.html) provides the following description:

> The data used here are from all 420 K-6 and K-8 schools from different districts in California, with data available for 1998 and 1999. Test scores are on the Stanford 9 standardized test administered to 5th-grade students. School characteristics and demographic variables for the students are averaged across the district. 

From all the variables contained in this dataset, we will use the following:

- `district`: district code (character type).
- `english`: percentage of students in the district who are English learners, i.e., students for whom English is a second language.
- `read`: average reading score by district, which comes from a standardized test administered to 5th-grade students.
- `math`: average math score.
- `students`: total enrollment.
- `teachers`: number of teachers.
- `calworks`: percent qualifying for CalWorks (income assistance).
- `lunch`: percent qualifying for reduced-price lunch.
- `computer`: number of computers.
- `expenditure`: expenditure per student.
- `income`: district average income (in USD 1,000).
- `english`: percent of English learners.

The math and reading scores are averaged into a new variable called `score`, which will be the response variable in this analysis

The number of students and teachers is combined into a student-to-teacher ratio, named `stratio`

In [ ]:
# Load the data
data(CASchools)

# Do some pre-processing of the data
CASchools_dat <- 
    CASchools %>%
    select(-district, -school, -grades, -county) %>%
    mutate(stratio = students/teachers,score=(math + read)/2)  %>% 
    mutate_if(is.numeric, round, 2)  %>% 
    select(-students, -teachers, -math, -read) 

head(CASchools_dat) 

**Question 1.1**
<br>{points: 1}

To explore a potential problem of multicollinearity, let's start by exploring (visually) the association between all potential input variables in the dataset. 

Remove the response variable `score` and use the plotting function `ggpairs`, from the library `GGally`, to generate pair plots of all the other variables in `CASchools_dat`. The `ggplot` object's name will be `CASchools_pair_plots`.

*Fill out those parts indicated with ..., uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# Adjust these numbers so the plot looks good in your desktop.
options(repr.plot.width = 15, repr.plot.height = 12) 

# CASchools_pair_plots <- 
#   ... %>%
#   select(- ...) %>% 
#   ...(progress = FALSE) +
#   theme(text = element_text(size = 15))

# your code here
fail() # No Answer - remove if you provide an answer

CASchools_pair_plots

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

Based on the pairwise plots and correlation coefficients in `CASchools_pair_plots`, do any pair of *input variables* appear to be highly positively or negatively correlated? Which ones?

Recall that a correlation coefficient equal to $1$ means a perfect positive linear association and $-1$ is a perfect negative linear association. For the purposes of this question, consider a correlation coefficient greater than 0.6 in absolute value "high".

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.3**
<br>{points: 1}

Another way to visualize pairwise correlation coefficients between all input variables is to use a heatmap. 

Creating this visualization will require prior data wrangling of `CASchools_dat`. 

We will first create a *melted* correlation matrix with all pairwise correlations between the input variables of `CASchools_dat` and name it `corr_matrix_CASchools`. You can use the function `cor()` for this purpose.
Recall to remove `score` from the dataset to visualize only the correlation between input variables.

The code below will generate a data frame called `corr_matrix_CASchools` with three columns:

- `var1`: first input variable.
- `var2`: second input variable.
- `corr`: correlation coefficient between `var1` and `var2`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# corr_matrix_CASchools <- 
#    CASchools_dat %>%
#    select(- ...) %>% 
#    ...() %>%
#    as_tibble(rownames = 'var1') %>%
#    pivot_longer(-var1, names_to = "var2", values_to = "corr")

# your code here
fail() # No Answer - remove if you provide an answer

corr_matrix_CASchools

In [ ]:
test_1.3()

**Question 1.4**
<br>{points: 1}

To create a proper $5 \times 5$ correlation matrix on top of a heatmap using `corr_matrix_CASchools`, the function `geom_tile()` from `ggplot2` is necessary. Create a plot called `plot_corr_matrix_CASchools` with the following characteristics:

- It has the style of a heatmap where the correlation coefficient gives the colour scale. You have to use `scale_fill_distiller()` to select the colour palette (`YlOrRd`, preferably) and adjust the correlation limits on the scale (between `-1` and `1`).
- The scale legend's title has to be indicated as **Correlation Coefficient**.
- Include the correlations from `corr_matrix_CASchools` in the corresponding matrix cells **rounded to 2 decimal places**. Use `geom_text()`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# Adjust these numbers so the plot looks good in your desktop.
options(repr.plot.width = 8, repr.plot.height = 5) 

# plot_corr_matrix_CASchools <- 
#   corr_matrix_CASchools %>%
#   ggplot(aes(..., ...)) +
#   ...(aes(fill = ...), color = "white") +
#   ...("Correlation Coefficient \n",
#       palette =  ...,
#       direction = 1, 
#       limits = ...
#     ) +
#     labs(x = "", y = "") +
#     theme_minimal() +
#     theme(
#         axis.text.x = element_text(angle = 45, vjust = 1, size = 14, hjust = 1),
#         axis.text.y = element_text(vjust = 1, size = 14, hjust = 1),
#         legend.title = element_text(size = 18),
#         legend.text = element_text(size = 12),
#         legend.key.size = unit(1.5, "cm")
#     ) +
#     coord_fixed() +
# #   geom_text(aes(..., ..., label = round(..., ...)), color = "black", size = 6)

# your code here
fail() # No Answer - remove if you provide an answer

plot_corr_matrix_CASchools

In [ ]:
test_1.4()

**Question 1.5**
<br>{points: 1}

Let's examine how the multicollinearity problem affects the least squares estimators. Using `CASchools_dat`, estimate an MLR called `MLR_CASchools` relating the response `score` to all the input variables selected.

*Tip*: in `lm`, you can use `.` at the RHS to avoid typing all input variables

> lm(response ~ ., data)

Obtain the estimated coefficients, standard errors, statistics and $p$-values using `tidy()`. Store the results in `MLR_CASchools_results`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# MLR_CASchools <- ...(..., ...)

# MLR_CASchools_results <- 
#    ...(...) %>% 
#    mutate_if(is.numeric, round, 2)

# your code here
fail() # No Answer - remove if you provide an answer

MLR_CASchools_results

In [ ]:
test_1.5()

**Question 1.6**
<br>{points: 1}

Some of the input variables are highly correlated and will be a problem for the least square estimates of the regression coefficients and their standard errors. A problem known as *multicollinearity*.

We can use the Variance Inflation Factor (VIF) to help quantify multicollinearity in a dataset. VIF ranges from $1$ (no multicollinearity) to infinity. The smaller the VIF, the lower the sign of multicollinearity. 

Calculate the VIF for your model `MLR_CASchools`. Assign the results to an object `VIF_MLR_CASchools`.

> **Hint:** A `vif()` function is available as part of the `car` package.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# VIF_MLR_CASchools <- ...(...)

# your code here
fail() # No Answer - remove if you provide an answer

round(VIF_MLR_CASchools, 3)

In [ ]:
test_1.6()

**Question 1.7**
<br>{points: 1}

Based on the VIF results in `VIF_MLR_CASchools`, answer the following questions:

**1.7.0.** Select the top-2 variables with the highest VIFs?

**A.** `calworks`.

**B.** `lunch`.

**C.** `computer`.

**D.** `expenditure`.

**E.** `income`.

**F.** `english`.

**G.** `stratio`.

*Assign your answer to the object `answer1.7.0`. Your answer must be a single string indicating the correct options **in alphabetical order** and surrounded by quotes (e.g., `"FG"` indicates you are selecting these two options).*

**1.7.1.** Does this pair also have the highest absolute correlation from all the pairwise correlations shown in the heatmap `plot_corr_matrix_CASchools`?

**A.** Yes.

**B.** No.

*Assign your answer to an object called `answer1.7.1`. Your answer should be a single character surrounded by quotes.*


In [ ]:
# answer1.7.0 <- ...
# answer1.7.1 <- ...

# your code here
fail() # No Answer - remove if you provide an answer


In [ ]:
test_1.7.0()
test_1.7.1()

**Question 1.8**
<br>{points: 1}

One way to deal with multicollinearity is to remove input variables associated with the large VIFs and strong pairwise correlations. Follow the steps below to identify and remove variables with potential multicollinearity issues.

- Identify the pair of input variables with the largest absolute correlation (positive or negative) in `plot_corr_matrix_CASchools`.
- Using `VIF_MLR_CASchools`, examine the VIF of both variables identified above and discard the variable with the larger VIF.
- Use all input variables in `CASchools_dat` except the discarded variable. You should have **6 input variables** remaining.

Use `lm()` to estimate a reduced MLR using only the subset of 6 selected input variables, and call the fitted model `red_MLR_CASchools`.

Use `tidy()` to summarize results in `red_MLR_CASchools_results`

<font style='color: darkred'>*Hint: You can use `.` to use all the variables (except the response) in the RHS of the formula, as well as `-` to remove a variable you do not want.*</font>

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# red_MLR_CASchools <- lm(formula = ..., 
#                         data = CASchools_dat)

# red_MLR_CASchools_results <- 
#     ...(...) %>%
#     mutate_if(is.numeric, round, 2)

# your code here
fail() # No Answer - remove if you provide an answer

red_MLR_CASchools_results

In [ ]:
test_1.8()

**Question 1.9**
<br>{points: 1}

Compare the results of `MLR_CASchools_results` with those of `red_MLR_CASchools_results`. Comment on how the estimates and $p$-values for the coefficients of the remaining input variables have changed after removing a highly correlated variable from the data.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.10**
<br>{points: 1}

Now, obtain the VIFs with `red_MLR_CASchools`. Assign the results to an object `VIF_red_MLR_CASchools`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# VIF_red_MLR_CASchools <- ...(...)

# your code here
fail() # No Answer - remove if you provide an answer

round(VIF_red_MLR_CASchools, 3)

In [ ]:
test_1.10()

**Question 1.11**
<br>{points: 1}

James et al. (2013) in [*An Introduction to Statistical Learning*](https://www.statlearning.com/) (see Section 3.3.3 in Subsection Collinearity) indicate that, as a rule of thumb, a VIF value that exceeds 5 or 10 is an indicator of a  multicollinearity problem. 

Comparing the results of `VIF_MLR_CASchools` with those of `VIF_red_MLR_CASchools`, and following this recommendation, do you think that excluding the variable with the largest VIF in `VIF_MLR_CASchools` solved the multicollinearity issue? 

**A.** Yes.

**B.** No.

*Assign your answer to an object called `answer1.11`. Your answer should be a single character surrounded by quotes.*

In [ ]:
# answer1.11 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.11()

## 2. Causality and Confounders

In this section, we will explore the problem of confounders in causal inference using a simulation. 

Roughly speaking, in a framework where $X$ is the input variable and $Y$ is the response, another input variable $C$ confounds the $X$-$Y$ relationship if $C$ and $X$ are associated, and $C$ and $Y$ are associated. 

For example, suppose a sports company's marketing team has developed a new video for their TikTok ad. They want to know if this <u>new</u> ad will <u>increase</u> the ad engagement (which they will measure via the *ad dwell time* in seconds, i.e., a continuous response) compared to the *current* ad they are running in a given customer population.
Furthermore, let us make the framework more complex. Suppose that athletes in this population have larger dwell times and are more interested in company innovations. 

**Question 2.0**
<br>{points: 1}

Under this framework, what is the response $Y$?

**A.** Type of TikTok ad.

**B.** Ad dwell time.

**C.** Athlete (yes/no).

*Assign your answer to the object `answer2.0`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.0()

**Question 2.1**
<br>{points: 1}

What is the main input of interest $X$?

**A.** Type of TikTok ad.

**B.** Ad dwell time.

**C.** Athlete (yes/no).

*Assign your answer to the object `answer2.1`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.1 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.1()

**Question 2.2**
<br>{points: 1}

True or False?

If we regress the dwell time on the type of add, ignoring which customers are athletes, the results of the estimation may be counfounded.

*Assign your answer to an object called `answer2.2`. Your answer should be either "true" or "false", surrounded by quotes.*

In [ ]:
# answer2.2 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.2()

There is more than one way to deal with confounding factors if we want to detect a causal relationship between $X$ and $Y$. Depending on the nature of the study, one could do the following:

1. In the case of observational studies, stratification is a good strategy. It implies checking the association between $X$ and $Y$ within subgroups of observations that share common values for the confounder(s). 
2. In an experimental study, such as A/B testing, we would randomize each subject to a particular value of $X$. Then, we compare $Y$ across the different groups of $X$. 

<font style='color: darkred'>Note: The second strategy is the golden standard in causal inference. Nonetheless, it is not always practical or even ethical to randomly assign experimental units to experimental groups.</font>

### Simulation Design: customers

To explore the problem of confounding using the TikTok ad case, we will construct a synthetic population containing information about 1,000,000 customers, of which 50% are athletes.

**Question 2.3**
<br>{points: 1}

In this question, you will begin by building a tibble, called `pop_pool` that contains one row for each customer in the population that contains one row per customer, for a total of 1,000,000 (`pop_size`) rows.

We first randomly simulate the variable `athlete`, with two possible values: "Y" (athlete) or "N" (non-athlete), reflecting the population composition described above, and add it as a column to the tibble. 

For future reference, count and store the number of athletes in the population in an object called `N_athletes`.

> **Check**: The tibble should have 1,000,000 rows and a single character column.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123) # DO NOT CHANGE!

# pop_size <- ...

# pop_pool <- tibble(
#   athlete = sample(c(rep("Y", ...), rep("N", ...))),
# )
#
# head(pop_pool)
#
# N_athletes <- sum(...)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.3()

### Simulation Design: dwell times

For each customer in the population, we will simulate two potential dwell times $Y$: one assuming the customer watches the current ad, and one assuming they watch the new ad. Thus, for each row in the tibble representing a customer, we will have a pair of "potential" responses. Later, only one of these potential outcomes will be assigned to each customer as the *realized response*.

Both potential responses (dwell times) will be generated from known distributions with different means (centers) to reflect each customer's sport experience (athlete vs non-athlete). Specifically, dwell times (in seconds) will be generated from a Normal distribution with a fixed standard deviation of 1, and means determined by design as follows:

- For *non-athlete* customers watching the *current ad*, the mean dwell time is 15 seconds.

- For *athlete* customers watching the *current ad*, the mean dwell time is 15 + 5 seconds, reflecting greater interest in the company's products.

- For *non-athlete* customers watching the *new ad*, the mean dwell time is 23 seconds, reflecting the improvement of the new ad.

- For *athlete* customers watching the *new ad*, the mean dwell time is 23 + 5 seconds.

We aim to estimate the causal effect of the *new ad on dwell time*, which, by design, is known and equals an increase of 8 seconds.


> **Note**: Because you fully control the data-generating process, all group means and the true "treatment" effect (i.e., effect of new ad on dwell times) are known by design. 

**Question 2.4**
<br>{points: 1}

Let's now simulate data! In the code below, use the function `rnorm()` to generate random values from a Normal distribution. 

> `rnorm()` has three arguments: n, mean, sd

Using the mean values specified above and a fixed standard deviations of 1, generate the variables `y_current_ad` and `y_new_ad`.

Use the indicator (`athlete == "Y"`), which equals 1 for athletes and 0 otherwise, to increase the mean dwell time for athletes in the population.

> **Check**: The tibble should have 1,000,000 rows and three columns: `athlete`, `y_current_ad`, and `y_new_ad`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*


In [ ]:
set.seed(123) # DO NOT CHANGE!

# pop_pool <-
#     pop_pool %>%
#     mutate(y_current_ad = rnorm(pop_size, mean = ... + ... * (athlete == "Y"), sd = ...),
#            y_new_ad = rnorm(pop_size, mean = ... + ... * (athlete == "Y"), sd = ...))

# your code here
fail() # No Answer - remove if you provide an answer

head(pop_pool)

In [ ]:
test_2.4()

### Observational data design

Now that we have generated potential outcomes and characteristics for all customers in the populations, we can generate a sample with the characteristics of an observational study. 

In observational studies customers choose the type of ad they watch, rather than being randomly assigned by the experimental designer. To mimic this self-selection, we will use the function `rbinom()` with `size = 1` to generate values for a variable called `x_self_choice` from a Bernoulli distribution. 

A Bernoulli distribution randomly generates values of 0 or 1 according to a specified probability. In this context, we can think of a value of 1 as indicating that the customer chooses the new ad, and a value of 0 as indicating that the customer chooses the current ad. We can specify the probability of generating a 1 to be higher for athletes than for non-athletes.

> `rbinom()` has three arguments: n, size, prob

Run the following cell to create a factor variable called `x_self_choice`, with values 0 and 1 as described above, where the probability of generating a value of 1 (corresponding to choosing the new ad) is higher for athletes than for non-athletes. Note that the factor levels in `x_self_choice` are labelled as `Current` for `0` and `New` for `1`.

> **Check**: The tibble should have 1,000,000 rows and four columns: `athlete`, `y_current_ad`, `y_new_ad`, and `x_self_choice`.

In [ ]:
# Run this cell before continuing.
set.seed(123)

pop_pool <-
    pop_pool %>% 
    mutate(x_self_choice = factor(rbinom(pop_size, size = 1, prob = 0.5 + 0.3 * (athlete == "Y")), 
                                  labels = c("Current", "New")))
           
head(pop_pool)

**Question 2.5**
<br>{points: 1}

We will now use the information stored in `pop_pool` to create a new variable, `y_obs`, containing each customer's *realized response* (i.e., the outcome actually observed in the observational study), based on `x_self_choice`. In other words, if the customer chooses to watch the new ad (that is, `x_self_choice = 1`), then `y_obs = y_new_ad`; otherwise, `y_obs = y_current_ad`.

Then, use the function `rep_sample_n()` to collect one sample of size `1000` from `pop_pool`, and call it `sample_TikTok`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123) # DO NOT CHANGE!

# pop_pool <-
#     pop_pool %>%
#     ...(y_obs = if_else(..., ..., ...))

# sample_TikTok <- ...(pop_pool, ...)

# your code here
fail() # No Answer - remove if you provide an answer

head(sample_TikTok)

In [ ]:
test_2.5()

**Question 2.6**
<br>{points: 1}

Let's compare the observed distributions and spread of dwell times for the two ads within `sample_TikTok` using side-by-side boxplots. 

Use `ggplot()` and appropriate geom to generate boxplots, and call the resulting plot `obs_dwell_time_boxplots`. Use the `stat_summary()` function to add the treatment means to each boxplot.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# Adjust these numbers so the plot looks good in your desktop.
options(repr.plot.width = 10, repr.plot.height = 7) 

# obs_dwell_time_boxplots <- 
#     ... %>%
#     ggplot() +
#     ...(aes(..., ..., fill = ...)) +
#     theme(text = element_text(size = 22)) +
#     ggtitle(...) +
#     xlab(...) +
#     ylab(...) +
#     stat_summary(aes(..., ..., fill = ...),
#         fun = ..., 
#         colour = "yellow",
#         geom = "point",
#         shape = 18,
#         size = 5
#     )

# your code here
fail() # No Answer - remove if you provide an answer

obs_dwell_time_boxplots


In [ ]:
test_2.6()

### Observational Data and Confounding

Our goal is to estimate the population-level effect of the new ad on dwell time compared to the current ad. From the simulation design, we know that this true effect equals an increase of 8 seconds. 

In an observational study, we would typically observe each customer’s dwell time and the ad they chose to watch, but we may not observe whether the customer is an athlete. However, because athlete status affects both ad choice and dwell time, ignoring `athlete` introduces confounding.

Thus, although the variable `athlete` is present in the population and in `sample_TikTok` by design, we create a new sample excluding the variable `athlete` to mimic an observational study with an unobserved confounding factor. In the analysis below, we will see how this omission can lead to a biased estimate of the true effect, even though the effect is known.

**Question 2.7**
<br>{points: 1}

Create a new object called `sample_obs_TikTok`, which excludes the variable `athlete` from `sample_TikTok`. 

Use the reduced sample to estimate a linear regression called `conf_obs_study_TikTok` of the dwell time (`y_obs`) versus the chosen type of ad (`x_self_choice`).  

Report the estimated coefficients, their standard errors, and corresponding $p$-values using `tidy()`. Include the corresponding asymptotic 95% confidence intervals. Store the results in the variable `conf_obs_study_TikTok_results`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# sample_obs_TikTok <- ... %>% 
#    ...(-...)

# conf_obs_study_TikTok <- ...(..., ...)

# conf_obs_study_TikTok_results <- 
#     ...(..., ...) %>%
#     mutate_if(is.numeric, round, 2)


# your code here
fail() # No Answer - remove if you provide an answer

conf_obs_study_TikTok_results

In [ ]:
test_2.7()

**Question 2.8**
<br>{points: 1}

Based on your output in `conf_obs_study_TikTok_results`, what can we conclude on the estimation of the population effect of the new ad?

**A.** The estimated effect in dwell time when switching from the current to the new ad increases by 9.83 seconds. The effect is largely overestimated in this analysis.

**B.** The estimated effect in dwell time when switching from the current to the new ad increases by 9.83 seconds. The effect is largely underestimated in this analysis.

**C.** The estimated effect in dwell time when switching from the new to the current ad increases by 9.83 seconds. The effect is largely overestimated in this analysis.

**D.** The estimated effect in dwell time when switching from the new to the current ad increases by 9.83 seconds. The effect is largely underestimated in this analysis.

*Assign your answer to an object called `answer2.8`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer2.8 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.8()

**Question 2.9**
<br>{points: 1}

Now, let us suppose we can collect additional information about the customers. In particular, we now use `sample_TikTok` with information about the sports experience of the customers contained in the variable `athlete`.

Estimate an MLR called `MLR_obs_study_TikTok` of the dwell time (`y_obs`) versus the type of ad and including the confounding variable. Report the estimated coefficients, their standard errors, and corresponding $p$-values using `tidy()`. Include the corresponding asymptotic 95% confidence intervals. 

Store the results in the variable `MLR_obs_study_TikTok_results`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# MLR_obs_study_TikTok <- ...(..., ...)

# MLR_obs_study_TikTok_results <- 
#     ...(..., ...) %>%
#     mutate_if(is.numeric, round, 2)

# your code here
fail() # No Answer - remove if you provide an answer

MLR_obs_study_TikTok_results

In [ ]:
test_2.9()

**Question 2.10**
<br>{points: 1}

Based on your output in `MLR_obs_study_TikTok_results`, how does the estimation of the effect of the new ad change when the confounder is included in the model?

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

<font color="darkred"> In observational studies, omitting a confounding factor affects the regression estimates, and causality can not be established. </font>

Including the confounder in the regression solves the problem. However, confounding variables are not always easy to collect (or even known). 

### Experimental data

Finally, let us imagine we can run an experimental study (i.e., A/B testing) in which customers from the `sample_TikTok` are randomly assigned an ad using a balanced design. 

**Question 2.11**
<br>{points: 1}

To simulate this case, we now create a two-level factor-type column called `x_randomized` by randomly assigning each customer to view either the current ad or the new ad. 

Using the function `if_else()`, create another column in `sample_TikTok` called `y_exp`, representing the obtained experimental response based on the add assigned by `x_randomized`. In other words, if the customer is assigned to watch the new ad, then `y_exp = y_new_ad`; otherwise, `y_exp = y_current_ad`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(1234) # DO NOT CHANGE!

# sample_TikTok <-
#     sample_TikTok %>%
#     mutate(x_randomized = ...(rep(c("Current","New"), ...)),
#            y_exp = ...(... == "Current", ..., ...))


# your code here
fail() # No Answer - remove if you provide an answer

head(sample_TikTok)

In [ ]:
test_2.11()

**Question 2.12**
<br>{points: 1}

As before, let's compare the experimental distributions and spread of dwell times for the two ads within `sample_TikTok` via side-by-side boxplots. 

Use `ggplot()` and appropriate geom to generate boxplots, and call the resulting plot `exp_dwell_time_boxplots`. Use the `stat_summary()` function to add the treatment means to each boxplot.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# exp_dwell_time_boxplots <- ... %>%
#   ggplot() +
#   ...(aes(..., ..., fill = ...)) +
#   theme(text = element_text(size = 22)) +
#   ggtitle(...) +
#   xlab(...) +
#   ylab(...) +
#   stat_summary(aes(..., ..., fill = ...),
#                fun = ...,
#                colour = "yellow",
#                geom = "point",
#                shape = 18,
#                size = 5
#   )

# your code here
fail() # No Answer - remove if you provide an answer

plot_grid(obs_dwell_time_boxplots, exp_dwell_time_boxplots)

In [ ]:
test_2.12()

**Question 2.13**
<br>{points: 1}

Create a new object called `sample_exp_TikTok`, which includes only the experimental dwell time (`y_exp`) and the assigned type of ad (`x_randomized`) from `sample_TikTok`.

Using `sample_exp_TikTok`, estimate the linear regression of the experimental dwell time (`y_exp`) versus the assigned type of ad (`x_randomized`), and call it `exp_study_TikTok`. 

Report the estimated coefficients, their standard errors, and corresponding $p$-values using `tidy()`. Include the corresponding asymptotic 95% confidence intervals. Store the results in the variable `exp_study_TikTok_results`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
#sample_exp_TikTok <- ... %>% 
#    select(..., ...)

# exp_study_TikTok <- ...(..., ...)

# exp_study_TikTok_results <- 
#     ...(..., ...) %>%
#     mutate_if(is.numeric, round, 2)

# your code here
fail() # No Answer - remove if you provide an answer

exp_study_TikTok_results

In [ ]:
test_2.13()

**Question 2.14**
<br>{points: 1}

Based on your output in `exp_study_TikTok_results`, which comes from the experimental study, what can we conclude about estimating the new ad's population effect?


**A.** The estimated effect in dwell time when switching from the new to the current ad increases by 8.03 seconds. A randomized experiment provides an accurate effect estimate without including additional variables in the model.

**B.** The estimated effect in dwell time when switching from the current to the new ad increases by 8.03 seconds. A randomized experiment provides an accurate effect estimate without including additional variables in the model.

**C.** The estimated effect in dwell time when switching from the new to the current ad increases by 8.03 seconds. The effect is largely overestimated in this analysis.

**D.** The estimated effect in dwell time when switching from the current to the new ad increases by 8.03 seconds. The effect is largely underestimated in this analysis.

*Assign your answer to an object called `answer2.14`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer2.14 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.14()

**Question 2.15**
<br>{points: 1}

Make a summary (it can be bullet points) of what you have learned from the simulation study in Section 2.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.